# Lesson 5 - Panorama

## Connect via Panorama

Panorama makes connecting to firewalls and sharing configuration between firewalls much easier. In all previous lessons you connected directly to a live firewall. In this lesson, you'll make changes on the firewall just like you did in previous lessons, but without ever needing the firewall's IP, username, or password, because Panorama will facilitate the connection the firewall for you. Device Framework makes this easy by simply acting according to the structure of the configuration tree.

When Device Framework is told to operate on a resource on a live firewall, Device Framework will make a connection to the nearest device in the configuration tree with a hostname/IP set. In all previous lessons, this was a Firewall object at the root of the tree. But let's look at an example where the root object is not a Firewall, and the Firewall has no login information:

In [ ]:
# This time, import the 'panorama' module, too
from pandevice import panorama, firewall, objects

In [ ]:
# Create a config tree with Panorama at the root
pano = panorama.Panorama('10.30.11.5', 'admin', 'Ignite18')
fw = firewall.Firewall(serial='015351000001577')
obj = objects.AddressObject('Server5', '5.5.5.5', description='Created on FW via Panorama')
pano.add(fw)
fw.add(obj)
pano.tree()

Notice in this case the Firewall object has no login information. No IP, username, or password. We created it with a serial number instead, so that Panorama knows which firewall this Firewall object represents.

Now, create the address object on the firewall.

In [ ]:
obj.create()

Use the firewall GUI to verify the address object was created. In order to create the address object on the firewall, Device Framework connected to Panorama, and Panorama connected to the firewall. When `obj.create()` was called, the Panorama object was the closes device to `obj` in the tree that had a hostname, so Device Framework made a connection to Panorama, not the firewall. If you added a hostname and credentials to the Firewall object, then the connection would be made to the firewall.

The net effect of this technique is the same as if Device Framework connected to the firewall directly, but this way you don't have to store the credentials of every firewall. As long as you know the credentials of Panorama and the serial of the firewall you are targeting, you can connect through Panorama but act as if you're directly connected to the firewall.

## Connection vs Scope

In the last section you connected to Panorama to create an address object, but the **scope** of the address object was still the Firewall, not Panorama. The address object was not created on Panorama, because Panorama only facilitated the connection. This decoupling of connection point and scope is very powerful. You define where to connect, and the scope of the object, with the structure of the configuration tree.

Next, we'll explore another possible object scope: Device Groups

## Device Groups

Device Groups are specific to Panorama and are a collection of policies and objects that are shared across multiple firewalls. In previous lessons, you placed policies and objects under a Firewall object in the configuration tree, but you can place them under a DeviceGroup object instead. Any policies and objects under a DeviceGroup object will be shared across any Firewall objects also under that DeviceGroup object.

Let's look at an example:

In [ ]:
# Import the needed modules and create a fresh Panorama object
from pandevice import panorama, firewall, objects
pano = panorama.Panorama('10.30.11.5', 'admin', 'Ignite18')

In [ ]:
# Use a special method to refresh all firewalls and device-groups on Panorama at once
pano.refresh_devices(add=True)
pano.tree()

**WAIT**, that cell might take a few seconds to finish. Hold here until the config tree appears.

In [ ]:
# Put the firewall and device group into variables so we can move them around
fw = pano.find('015351000001577', firewall.Firewall)
dg = pano.find('FW01-Group')

In [ ]:
# Remove the firewall from Panorama, and add it to the Device Group
pano.remove(fw)
dg.add(fw)

In [ ]:
pano.tree()

In [ ]:
# Now create some AddressObjects
obj1 = fw.add(objects.AddressObject('Firewall-scope-object', '1.1.1.1'))
obj2 = dg.add(objects.AddressObject('DeviceGroup-scope-object', '2.2.2.2'))
pano.tree()

In [ ]:
# Apply all the changed objects
fw.create()
obj1.create()
obj2.create()

Notice we had to call `create()` on the `fw` Firewall object. That's because it also changed, it was moved into a device group. So essentially this is creating the firewall inside the device group. To move it to another device group, you would first `delete()` it from this device group, then `create()` it in another.

Verify everything worked by checking the Panorama and Firewall GUI's.  Under **Objects -> Addresses** you should see the **Firewall-scope-object** on the firewall and the **DeviceGroup-scope-object** on the Panorama inside the FW01-Group device group.

## Scope Example

There are many scopes that an object can have, here are a few examples of places you can put objects:

- Firewall - shared
- Firewall - in a specific vsys
- Panorama - shared
- Panorama - in a specific device group
- Panorama - in a specific template

Each one of these scopes can be represented in the configuration tree by placement of an object in a specific place in the config tree that represents that scope.

This example demonstrates placing an object in each one of the above scopes using a single configuration tree:

In [ ]:
from pandevice import panorama, firewall, network, objects, device
# Create scopes: Panorama, Firewall, Device Group, Vsys, and Template
pano = panorama.Panorama('10.30.11.5', 'admin', 'Ignite18')
dg = panorama.DeviceGroup('MyDeviceGroup')
fw = firewall.Firewall(serial='015351000001577')
vsys = device.Vsys('vsys2', 'MyVsys')
template = panorama.Template('MyTemplate')

# Create objects to place in different scopes
obj1 = objects.AddressObject('Firewall-shared', '1.1.1.1')
obj2 = objects.AddressObject('Firewall-in a specific vsys', '1.1.1.1')
obj3 = objects.AddressObject('Panorama-shared', '1.1.1.1')
obj4 = objects.AddressObject('Panorama-in a specific device group', '1.1.1.1')
obj5 = network.Zone('Panorama-in a specific template')

# Build configuration tree
pano.add(dg)
pano.add(template)
dg.add(fw)
fw.add(vsys)

# Add objects to different scopes in configuration tree
fw.add(obj1)
vsys.add(obj2)
pano.add(obj3)
dg.add(obj4)
template.add(obj5)

# Show the configuraiton tree
pano.tree()

This is a complex example and not really functional, so don't worry if you don't fully understand it. Consider this as a reference and an example of the possibilities.

## Commit and Push

In Lesson 3 you learned how to perform a commit on a firewall. Commit to firewalls from Panorama is a 2 step process. First, you commit the changes to Panorama itself. Then, push the changes to the firewalls. This second step is called a 'push' in the Panorama GUI, but it's called a 'commit-all' in the API. Different terms, but same action.

Let's commit the changes to the device group and firewall. This cell is performing 2 commits so **it might take a while**.

In [ ]:
commit_result = pano.commit(sync=True)
commitall_result = pano.commit_all(sync=True, devicegroup='FW01-Group')

# Print the results of the commits
from pprint import pformat
print("Commit result: {}".format(pformat( commit_result )))
print("Commit-all result: {}".format(pformat( commitall_result )))

Examine the return values of the commit commands. The first commit is on Panorama only, so it has one result field. The second commit is actually a push to all the devices in the FW01-Group device group, so you get a result for the commit as a whole which indicates the commit initiated, and you get a result, errors, and warnings for each individual device by serial number. This is useful for programatically tracking down commit failures on specific devices and responding with a remediation or rollback.

## Next steps

There is a lot more power and possibility in using Panorama with Device Framework than is covered in this lesson, but the tools you learned here will help you explore the Device Framework API reference to learn other powerful techniques for configuration management across multiple firewalls.

Next, you'll learn how to check the state and health of a firewall in [Lesson 6 - Operational Commands](Lesson 6 - Operational Commands.ipynb).